#Настройка

Присвоить переменной "hours" значение 12, 24 или 36

In [ ]:
hours = 24

In [ ]:
hours_button_map = {12: 'id_l_1', 24: 'id_l_2', 36: 'id_l_3'}
try:
    hours_button_id = hours_button_map[hours]
except:
    raise KeyError('Неправильное значение переменной "hours"')

In [ ]:
from google.colab import files
filename = files.upload()
filename = next(iter(filename))

Saving rt_08-07-2021_12.xlsx to rt_08-07-2021_12.xlsx


#Скрипт

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
!pip install requests-html
!pip install xlutils

     |████████████████████████████████| 911kB 14.9MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubun

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
#!/usr/bin/env python3
import collections
from typing import Dict, List
from collections import defaultdict
from openpyxl.worksheet.worksheet import Worksheet
import sys
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib3
import requests
import openpyxl

import re
import warnings
import requests
from openpyxl import load_workbook
from datetime import datetime

warnings.filterwarnings('ignore')
urllib3.disable_warnings()


URLS = [
    'https://meteoinfo.ru/hdmapsmeteoalarmszfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmcfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmprfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyugfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmyskfo',
    'https://meteoinfo.ru/hdmapsmeteoalarmural',
    'https://meteoinfo.ru/hdmapsmeteoalarmsyb',
    'https://meteoinfo.ru/hdmapsmeteoalarmdv',
]

HAZARDS_URL = 'https://meteoinfo.ru/hazardsbull'

SIZE = 4

SHEET_NAME = 'Билет по регионам'

REGION_COLUMN = 2

FORECAST_COLUMNS = (6, 7, 8, 9)
FACTOR_COLUMNS = (10, 11, 12, 13)

FORCASTS = (
    'Угроза повреждения ЛЭП и линий связи, обрушения слабо укрепленных, широкоформатных, ветхих и рекламных конструкций',
    'Угроза выхода из строя систем ЖКХ, городских коммуникаций',
    'Угроза затопления населенных пунктов, дорог, мостов',
    'Угроза сбоев в работе всех видов транспорта, увеличение количества ДТП',
    'Угроза жизни и здоровью населения',
    'Увеличение количества травм пешеходов',
    'Угроза повреждения населенных пунктов, дорог, мостов'
)

# INTERCHANGEABLE_FACTORS = [
#     {'Ветер', 'Дождь', 'Снег/Обледенение', 'Гроза'},
#     {'Заморозки', 'Очень низкая температура'},
#     {'Туман', 'Голеледно - изморозевые отложения'},
#     {'Пожарная опасность'},
#     {'Паводок'},
#     {'Очень высокая температура'}
# ]

FORCAST_MAP = {
    'очень высокая температура': set((FORCASTS[4],)),
    'очень низкая температура': set((FORCASTS[1], FORCASTS[4])),
    'заморозки': set((FORCASTS[1], FORCASTS[4])),
    'ветер': set((FORCASTS[0], FORCASTS[1], FORCASTS[4])),
    'дождь': set((FORCASTS[0], FORCASTS[1])),
    'гроза': set((FORCASTS[0], FORCASTS[1])),
    'град': set((FORCASTS[0], FORCASTS[1])),
    'паводок': set((FORCASTS[2], FORCASTS[4])),
    'наводнение': set((FORCASTS[2], FORCASTS[4])),
    'снег/Обледенение': set((FORCASTS[0], FORCASTS[1], FORCASTS[3])),
    'голеледно - изморозевые отложения': set((FORCASTS[3], FORCASTS[5])),
    'лавины': set((FORCASTS[4], FORCASTS[6])),
    'сель': set((FORCASTS[4], FORCASTS[6])),
    'прибрежные события': set((FORCASTS[1], FORCASTS[6], FORCASTS[4])),
    'пыльная (песчаная) буря': set((FORCASTS[0], FORCASTS[1], FORCASTS[3], FORCASTS[4])),
    'туман': set((FORCASTS[3],)),
    'пожарная опасность': set((FORCASTS[6], FORCASTS[4])),
    'прочие опасности': set(),
}

DASHBOARD_LABEL_MAP = {
    'очень высокая температура': 'Критическое повышение температуры воздуха',
    'очень низкая температура': 'Критическое понижение температуры воздуха',
    'заморозки': 'Заморозки',
    'ветер': 'Сильный ветер',
    'дождь': 'Дождь',
    'гроза': 'Гроза',
    'паводок': 'Паводок, подтопление',
    'наводнение': 'Наводнение',
    'снег/Обледенение': 'Снегопад, метель',
    'голеледно - изморозевые отложения': 'Гололедица',
    'лавины': 'Оползни, обвалы, снежные лавины',
    'сель': 'Сель',
    'прибрежные события': 'Прибрежные события',
    'пыльная (песчаная) буря': 'Пыльная (песчаная) буря',
    'туман': 'Туман',
    'пожарная опасность': 'Пожароопасная обстановка, пожары',
    'прочие опасности': 'Прочие опасности',
    'град': 'Град или снежный дождь'
}

FACTORS = FORCAST_MAP.keys()

regions_need_to_join = {'саха',
                        'магаданская',
                        'ямало ненецкий ао',
                        'камчатский край',
                        'амурская',
                        'хабаровский край',
                        'чеч',
                        'северная осетия',
                        'красноярский',
                        'моск'}


def join_region(region: str) -> str:
    if region.startswith('сахалин'):
        return 'сахалин'
    for reg in regions_need_to_join:
        if region.startswith(reg):
            return reg
    return region


def normolize_string(string: str):
    string = string.strip().lower().translate({ord('.'): None})
    string = ' '.join(string.split('-'))

    # remove stop words
    stop_words = ['г', 'обл', 'область', 'республика', 'югра', 'ао']
    try:
        splitted_string = string.split()
    except IndexError:
        return ''
    string = ' '.join(w for w in splitted_string if w not in stop_words)
    return join_region(string)


def remove_interchangeable_factors(factor_desc_list: list):
    factors_set = set()
    prev_len_factors_set = 0

    for factor_desc in factor_desc_list:
        factor = factor_desc[0]
        forecast = FORCAST_MAP.get(factor.lower())
        if not forecast:
            continue
        factors_set.update(forecast)
        if len(factors_set) == prev_len_factors_set:
            factor_desc_list.remove(factor_desc)
            return factor_desc_list
        prev_len_factors_set = len(factors_set)
    return factor_desc_list


def analyze_factors(factors_desc: list):
    final_forecasts = set()

    prev_len = len(factors_desc)
    while len(factors_desc) > SIZE:
        factors_desc = remove_interchangeable_factors(factors_desc)
        if len(factors_desc) == prev_len:  # If nothing has been deleted, means there is no interchangeable factors
            factors_desc = factors_desc[:SIZE]
        prev_len = len(factors_desc)

    for factor, desc in factors_desc:
        forecast: set = FORCAST_MAP.get(factor.lower())
        if forecast is None:
            print(f"\x1b[31mНЕИЗВЕСТНЫЙ ФАКТОР \"{factor}\"\x1b[0m")
            continue
        final_forecasts.update(forecast)

    while len(final_forecasts) > SIZE:
        final_forecasts.pop()

    return final_forecasts, set(factors_desc)


def factor_mapping(factor: str, description: str):
    dashboard_factor = DASHBOARD_LABEL_MAP.get(factor.lower())
    if dashboard_factor == 'Дождь':
        for word in description.split():
            if word.startswith('сильн') or word.startswith('лив'):
                dashboard_factor = 'Сильный дождь, ливневой дождь'
                break
    return dashboard_factor

def fill_excel_regions_weather(ws: Worksheet, regions: dict):
    for row in range(2, ws.max_row + 1):
        name = ws.cell(row=row, column=REGION_COLUMN).value
        if not name:
            continue

        name = normolize_string(name)
        factors_desc = regions.get(name)

        if not factors_desc: # if there is no forecast for the region
            # print(name)
            continue

        forecasts, factors_desc = analyze_factors(factors_desc)
        for column, forecast in zip(FORECAST_COLUMNS, forecasts):
            ws.cell(row=row, column=column).value = forecast

        for column, factor_desc in zip(FACTOR_COLUMNS, factors_desc):
            factor = factor_desc[0]
            desc = factor_desc[1]
            ws.cell(row=row, column=column).value = factor_mapping(factor, desc)


def get_hazards(url: str):
    session = HTMLSession(browser_args=["--proxy-server=201.249.161.51:999"])
    response = session.get(url, verify=False)
    rows = response.html.find('#div_1', first=True).xpath('*/table/tbody/tr')
    row_data = rows[1].xpath('//p', first=True)
    print(row_data.text)


def parse_description(factor_list: list, region_factor_desc: str):
    for word in re.split(';|,|\.| ', region_factor_desc):
        # try:
        #     factors = list(zip(*factor_list))[0]
        # except IndexError:
        #     factors = []
        if (word.lower().startswith('дожд') or word.lower().startswith('лив')):
            factor_list.append(('Дождь', region_factor_desc))
        elif word.lower().startswith('гроз'):
            factor_list.append(('Гроза', region_factor_desc))
        elif word.lower().startswith('град'):
            factor_list.append(('Град', region_factor_desc))
        elif (word.lower().startswith('шквал') or word.lower().startswith('вет')):
            factor_list.append(('Ветер', region_factor_desc))


def get_regions(urls: List[str]) -> Dict:
    # factors = defaultdict(list)
    # descriptions = defaultdict(list)
    regions = defaultdict(list)
    for url in urls:
        wd.get(url)
        hours24_button = wd.find_element_by_id(hours_button_id)
        hours24_button.click()
        for row in wd.find_elements_by_xpath('//div[@class="test"]'):
            try:
                row_data_bs = BeautifulSoup(row.get_attribute('data-original-title'),
                                            features='lxml')
            except:
                print('error')
                continue  # if raw_data is None
            region_name = normolize_string(row_data_bs.find(class_='cl_tbl_obl_name_tooltip').text)
            region_factor = row_data_bs.find(class_='cl_tbl_tooltip_text').find('b').text.strip('. ')
            region_factor_desc = '.'.join(row_data_bs.find(class_='cl_tbl_tooltip_text').text.split('.')[1:]).strip()

            if (region_factor == 'КМЯ' or region_factor == 'Прочие опасности'):
                parse_description(regions[region_name], region_factor_desc)
                # print(regions[region_name], region_factor)
                continue

            

            try:
                factors = list(zip(*regions[region_name]))[0]
            except IndexError:
                factors = []

            if region_factor != '':
                regions[region_name].append((region_factor, region_factor_desc))

    return regions


In [ ]:
def fill_excel_covid_rt(ws: Worksheet):
    obl_pattern = re.compile(r'\bобл\.', re.IGNORECASE)
    ao_pattern = re.compile(r'(?<!Еврейская)\bао\b', re.IGNORECASE)
    gorod_pattern = re.compile(r'\bг\.', re.IGNORECASE)
    strange_pattern = re.compile(r'[—]')

    url = 'https://yastat.net/s3/milab/2020/covid19-stat/data/v10/deep_data.json?v=1618916098074'
    sheet = 'Билет по регионам'

    json_dict = requests.get(url, verify=False).json()

    region_rt_dict = {strange_pattern.sub('-', x['info']['name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}
    region_rt_dict_short_name = {strange_pattern.sub('-', x['info']['short_name'].lower()): x['info'].get('rt') for x in json_dict['russia_stat_struct']['data'].values()}

    for row in range(2, ws.max_row + 1):
        # print(ws.cell(row=row, column=2).value)
        key = ws.cell(row=row, column=2).value.lower()

        key = obl_pattern.sub('область', key).strip()
        key = gorod_pattern.sub('', key).strip()

        value = region_rt_dict_short_name.get(key, region_rt_dict.get(ao_pattern.sub('автономный округ', key).strip()))

        ws.cell(row=row, column=3).value = round(value, 2) if value else value

In [ ]:
# import xlrd
# from openpyxl.workbook import Workbook
# from openpyxl.reader.excel import load_workbook, InvalidFileException

# def open_xls_as_xlsx(filename):
#     # first open using xlrd
#     book = xlrd.open_workbook(filename)
#     index = 0
#     nrows, ncols = 0, 0
#     while nrows * ncols == 0:
#         sheet = book.sheet_by_index(index)
#         nrows = sheet.nrows
#         ncols = sheet.ncols
#         index += 1

#     # prepare a xlsx sheet
#     book1 = Workbook()
#     sheet1 = book1.get_active_sheet()

#     for row in range(1, nrows):
#         for col in range(1, ncols):
#             sheet1.cell(row=row, column=col).value = sheet.cell_value(row, col)

#     return book1

In [ ]:
# import openpyxl
# from xlrd import open_workbook
# from xlutils.copy import copy
# from io import BytesIO


# def save_xls(Wb_xlsx):
#     buffer = BytesIO()
#     Wb_xlsx.save(buffer) # xlsx format save
#     wb_xls = copy(open_workbook(file_contents=buffer.getvalue()))
#     wb_xls.save('a.xls')

In [ ]:
def clear_table(ws: Worksheet):
    for row in range(2, ws.max_row + 1):
        for column in (FORECAST_COLUMNS + FACTOR_COLUMNS + (3,)):
            ws.cell(row=row, column=column).value = None

In [ ]:


def main(filename: str):
    wb = openpyxl.load_workbook(filename)
    ws = wb[SHEET_NAME]

    clear_table(ws)
    fill_excel_covid_rt(ws=ws)
    regions = get_regions(urls=URLS)
    print(regions)
    # remove duplicate factors
    for region in regions:
        factors = regions[region]
        final_list = []
        for factor in factors:
            try:
                unique_factors = list(zip(*final_list))[0]
            except:
                unique_factors = []
            if factor[0] not in unique_factors:
                final_list.append(factor)
        regions[region] = final_list


    fill_excel_regions_weather(ws=ws, regions=regions)
    get_hazards(HAZARDS_URL)

    today = datetime.today().strftime("%d-%m-%Y")
    splitted_filename = filename.split('.')
    splitted_filename[0] = f'{splitted_filename[0]}_{today}_{hours}'
    output_file_name = '.'.join(splitted_filename)
    wb.save(output_file_name)
    return output_file_name

# Результат

In [ ]:
out_file = main(filename)

defaultdict(<class 'list'>, {'архангельская': [('Пожарная опасность', 'Период предупреждения до 21 ч. 12 июля местами 5 класс пожароопасности (более 3000 градусов по формуле Нестерова)'), ('Ветер', 'Период предупреждения до 21 ч. 12 июля при грозе порывы 15-20 м/с'), ('Гроза', 'Период предупреждения до 21 ч. 12 июля местами гроза')], 'вологодская': [('Очень высокая температура', 'Период предупреждения до 21 ч. 13 июля местами температура +30 градусов и более; аномально-жаркая погода'), ('Гроза', 'Период предупреждения с 21 ч. 12 июля до 21 ч. 13 июля Местами гроза'), ('Пожарная опасность', 'Период предупреждения до 21 ч. 13 июля местами пожароопасность 4 кл.')], 'калининградская': [], 'ленинградская': [('Пожарная опасность', 'Период предупреждения до 13 ч. 13 июля местами 4 класс'), ('Очень высокая температура', 'Период предупреждения до 13 ч. 13 июля аномально жаркая погода с отклонением среднесуточной температуры воздуха на 7-10 гр.от климатической нормы, местами по области +35 гр. и

In [ ]:
files.download(out_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# import pandas as pd

# wb = openpyxl.load_workbook(out_file)


# pd_wb = pd.read_excel(out_file, sheet_name=wb.sheetnames)
# xlsWriter = pd.ExcelWriter(out_file[:-1], engine = 'xlwt')

# pd_wb['Инцидент'].to_excel(xlsWriter, sheet_name='Инцидент', index=False)
# pd_wb['Билет по регионам'].to_excel(xlsWriter, sheet_name='Билет по регионам', index=False)
# xlsWriter.save()
